In [ ]:
import numpy as np
import pandas as pd

In [129]:
#Meses invierno verano, 1 es verano, 0 es invierno.
inv_verano = pd.read_excel('../02_data/verano.xlsx', sheet_name='veranos', skiprows=1)
inv_verano = pd.melt(inv_verano, id_vars=['zona'], value_vars=inv_verano.columns[1:] , var_name='mes_no', value_name='inv_verano')
inv_verano.head()

,zona,mes_no,inv_verano
0,chillan,1,1
1,constitucion,1,1
2,arauco,1,1
3,valdivia,1,1
4,chillan,2,1


In [79]:
import datetime
import dateutil.relativedelta as relativedelta

mes_inicio = 9
año_inicio = 2023

date_inicio = datetime.datetime(year=año_inicio, month=mes_inicio, day=10)

periodos = []
horizonte_planificacion = 12
for mes in range(0, horizonte_planificacion + 1):
    aux_per = date_inicio + relativedelta.relativedelta(months=mes)
    periodos.append(str(aux_per.year) + '-' + str(aux_per.month).zfill(2))

periodos = pd.DataFrame(periodos, columns=['periodos'])
periodos['año'] = periodos['periodos'].str[:4]
periodos['mes_no'] = periodos['periodos'].str[5:7].astype(int)
periodos = pd.merge(periodos, inv_verano, how='left', on='mes_no')
periodos['mes_planificado'] = periodos.index + 1
periodos.head()

,periodos,año,mes_no,zona,inv_verano,mes_planificado
0,2023-09,2023,9,chillan,0,1
1,2023-09,2023,9,constitucion,0,2
2,2023-09,2023,9,arauco,0,3
3,2023-09,2023,9,valdivia,0,4
4,2023-10,2023,10,chillan,1,5


In [ ]:
#Carga de plantas

base_plantas = pd.read_excel('../02_data/plantas.xlsx', sheet_name='plantas')
display(base_plantas.head())

#Lectura de parametros planta
parametros_planta = pd.read_excel('../02_data/plantas.xlsx', sheet_name='parametros_planta')
display(parametros_planta.head())

#Lectura consumo plantas
consumo_plantas = pd.read_excel('../02_data/plantas.xlsx', sheet_name='consumo_plantas')
consumo_plantas = pd.melt(consumo_plantas, consumo_plantas.columns[:2], consumo_plantas.columns[2:], var_name='mes_planificado', value_name='volumen_consumo')
display(consumo_plantas.head())


In [ ]:
#Carga Ingresos: Compras y produccion
base_ingresos = pd.read_excel('../02_data/ingresos.xlsx', sheet_name='ingresos')
base_ingresos.head()

In [ ]:
#Carga Ingreso: Volumen mensual
base_ingreso_vol = pd.read_excel('../02_data/ingresos_volumen.xlsx', sheet_name='ingresovolumen')
base_ingreso_vol.head()

In [126]:
rango_mes_tipo = pd.read_excel('../02_data/parametro_antiguedad.xlsx', sheet_name='rango_mes_tipo')
rangos_demanda = pd.read_excel('../02_data/parametro_antiguedad.xlsx', sheet_name='parametro_antiguedad')

rangos_demanda_ext = pd.DataFrame(columns=['id_tipo_rango', 'rango', 'antiguedad'])
for ix in rangos_demanda.index:
    for ant in range(rangos_demanda.at[ix, 'inicio_rango'], rangos_demanda.at[ix, 'fin_rango'] + 1):
        aux = [[rangos_demanda.at[ix, 'id_tipo_rango'], rangos_demanda.at[ix, 'rango'], ant ]]
        aux = pd.DataFrame(aux, columns=['id_tipo_rango', 'rango', 'antiguedad'])
        rangos_demanda_ext = pd.concat([rangos_demanda_ext, aux], ignore_index=True)

rangos_demanda_ext.head()

,id_tipo_rango,rango,antiguedad
0,1,0-3,0
1,1,0-3,1
2,1,0-3,2
3,1,0-3,3
4,1,4-6,4


In [ ]:
## Matriz de demandas.

#Consumo_planta_tipo + parametros_planta_tipo
#Calculo demanda
df = pd.merge(consumo_plantas, parametros_planta, on=['nombre_planta', 'producto'])
df.loc[df['mes_planificado'] != 1, 'stock_inicial'] = 0
df['demanda'] = df['volumen_consumo'] + df['stock_seguridad_planta'] - df['stock_inicial']

#Agregar los tipos de rango por mes
df = pd.merge(df, rango_mes_tipo, how='left', on=['nombre_planta', 'producto', 'mes_planificado'])
df = pd.merge(df, rangos_demanda, how='left', on='id_tipo_rango')
df['demanda_rango'] = df['demanda'] * df['porcentaje']
df = df[['nombre_planta', 'producto', 'mes_planificado', 'id_tipo_rango', 'rango', 'inicio_rango', 'fin_rango', 'demanda_rango']].dropna()
df['id_tipo_rango'] = df['id_tipo_rango'].astype(int)
df['inicio_rango'] = df['inicio_rango'].astype(int)
df['fin_rango'] = df['fin_rango'].astype(int)

done_demanda = df.copy()
done_demanda

In [67]:
### Carga de Almacenes ###
base_almacenes = pd.read_excel('../02_data/Almacenes.xlsx', sheet_name='almacenes')
no_almacenes = len(base_almacenes)

In [83]:
#Stock almacen EN MES 0
base_stock_almacenes = pd.read_excel('../02_data/stock_almacen.xlsx', sheet_name='cierrestock')
done_alm = pd.merge(base_almacenes, base_stock_almacenes, how='left', on='id_almacen')
done_alm = pd.merge()

,id_almacen,nombre_almacen,nombre_planta,producto,zona,id_almacen_acanche,carpeta,cosecha_compra,tipo_almacen,costo_existencia,capacidad,ant_meses,cierre_stock
0,alm1,Cancha compra EUNI,planta1,EUNI CC,chillan,N_A,N_A,compra,cancha,10,100,1,200.000000
1,alm1,Cancha compra EUNI,planta1,EUNI CC,chillan,N_A,N_A,compra,cancha,10,100,2,300.000000
2,alm1,Cancha compra EUNI,planta1,EUNI CC,chillan,N_A,N_A,compra,cancha,10,100,3,100.000000
3,alm1,Cancha compra EUNI,planta1,EUNI CC,chillan,N_A,N_A,compra,cancha,10,100,4,20.000000
4,alm1,Cancha compra EUNI,planta1,EUNI CC,chillan,N_A,N_A,compra,cancha,10,100,5,10.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,alm24,Bosque cosecha EUNI cc RIPIO,planta1,EUNI CC,chillan,alm12,ripio,cosecha,bosque,10,100,9,14259.432497
308,alm24,Bosque cosecha EUNI cc RIPIO,planta1,EUNI CC,chillan,alm12,ripio,cosecha,bosque,10,100,10,6627.735165
309,alm24,Bosque cosecha EUNI cc RIPIO,planta1,EUNI CC,chillan,alm12,ripio,cosecha,bosque,10,100,11,5673.772999
310,alm24,Bosque cosecha EUNI cc RIPIO,planta1,EUNI CC,chillan,alm12,ripio,cosecha,bosque,10,100,12,4719.810832


In [87]:
primer_periodo = str(date_inicio.year) + '-' + str(date_inicio.month).zfill(2)
base_stock_almacenes = base_stock_almacenes.rename(columns={'cierre_stock':primer_periodo})
base_stock_almacenes

,id_almacen,ant_meses,2023-09
0,alm1,1,200.000000
1,alm1,2,300.000000
2,alm1,3,100.000000
3,alm1,4,20.000000
4,alm1,5,10.000000
...,...,...,...
307,alm24,9,14259.432497
308,alm24,10,6627.735165
309,alm24,11,5673.772999
310,alm24,12,4719.810832


In [77]:
#Armar matriz de almacen-antiguedad-mes
almacenes = done_alm[['id_almacen']].drop_duplicates().reset_index(drop=True)
antiguedades = pd.DataFrame(range(1,13+1), columns=['antiguedad'])
periodoz = periodos[['periodos']].drop_duplicates().reset_index(drop=True)

almacenes = pd.merge(almacenes, antiguedades, how='cross')
almacenes = pd.merge(almacenes, periodoz, how='cross')
almacenes['stock'] = 0
almacenes = almacenes.pivot_table(index=['id_almacen', 'antiguedad'], columns=['periodos'], values='stock').rename_axis(columns=None).reset_index()
almacenes

,id_almacen,antiguedad,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09
0,alm1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,alm1,2,0,0,0,0,0,0,0,0,0,0,0,0,0
2,alm1,3,0,0,0,0,0,0,0,0,0,0,0,0,0
3,alm1,4,0,0,0,0,0,0,0,0,0,0,0,0,0
4,alm1,5,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,alm9,9,0,0,0,0,0,0,0,0,0,0,0,0,0
308,alm9,10,0,0,0,0,0,0,0,0,0,0,0,0,0
309,alm9,11,0,0,0,0,0,0,0,0,0,0,0,0,0
310,alm9,12,0,0,0,0,0,0,0,0,0,0,0,0,0


In [125]:
#Modifica demanda del  primer periodo de acuerdo a disponibilidad.

for planta in df['nombre_planta'].unique():
    for producto in df[df['nombre_planta'] == planta]['producto'].unique():
        for mes in [1]:
            aux = df[(df['nombre_planta'] == planta) & (df['producto'] == producto) & (df['mes_planificado'] == mes)]
            aux = aux.sort_values(by='fin_rango', ascending=False)
            for ix1 in aux.index:
                aux_id_rango = aux.at[ix1, 'id_tipo_rango']
                aux_rango = aux.at[ix1, 'rango']
                rango_aux = rangos_demanda_ext[rangos_demanda_ext['id_tipo_rango'] == aux_id_rango]

                new_alm = done_alm.copy()
                new_alm = pd.merge(new_alm, rango_aux, how='cross')
                new_alm = new_alm[new_alm['ant_meses'] == new_alm['antiguedad']]
                new_alm = new_alm[(new_alm['nombre_planta'] == planta) & (new_alm['producto'] == producto)]
                new_alm = new_alm.groupby(['nombre_planta', 'producto', 'rango'], as_index=False).agg(volumen=('cierre_stock', 'sum'))
                new_alm = new_alm[new_alm['rango'] == aux_rango].reset_index(drop=True)
                display(new_alm)

,nombre_planta,producto,rango,volumen
0,planta1,EUGL CC,7-9,32347.378956


,nombre_planta,producto,rango,volumen
0,planta1,EUGL CC,4-6,50709.751359


,nombre_planta,producto,rango,volumen
0,planta1,EUGL CC,0-3,23437.954566


In [123]:
display(aux)

,nombre_planta,producto,mes_planificado,id_tipo_rango,rango,inicio_rango,fin_rango,demanda_rango
38,planta1,EUGL CC,1,1,7-9,7,9,10.0
37,planta1,EUGL CC,1,1,4-6,4,6,20.0
36,planta1,EUGL CC,1,1,0-3,0,3,70.0
